### 1.Importing Necessary libraries

In [131]:
import pandas as pd
import numpy as np
import os
import pickle
from keras.optimizers import SGD, Adam, Nadam, RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence
from keras.models import Sequential,Model,load_model
from keras.layers import Embedding,Conv1D,MaxPooling1D
from keras.layers.core import Dense, Activation,Dropout ,Flatten
from keras.layers.recurrent import LSTM
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import sequence
from keras.preprocessing.text import text_to_word_sequence,one_hot,Tokenizer
from keras.constraints import maxnorm
from keras.callbacks import ModelCheckpoint,TensorBoard, ReduceLROnPlateau,EarlyStopping
from keras.applications import Xception
from keras import regularizers
from keras import backend as K
import keras
import numpy as np
import pandas as pd
import cv2
import os
import glob
import math
import matplotlib.pyplot as plt

### 2. Initialize the seed inorder to get the same output every single time

In [132]:
seed = 120
np.random.seed(seed)

### 3. Loading the training data

In [133]:
train_path = './data/train/train.csv'
train_df = pd.read_csv(train_path)# Loading a csv file with headers 
train_df.head()

,sentence,BookRestaurant,GetWeather,PlayMusic,RateBook
0,book The Middle East restaurant in IN for noon,1,0,0,0
1,Book a table at T-Rex distant from Halsey St.,1,0,0,0
2,I'd like to eat at a taverna that serves chili...,1,0,0,0
3,I have a party of four in Japan and need a res...,1,0,0,0
4,Please make a restaurant reservation for somew...,1,0,0,0


### 4. Shuffling the dataframe

In [134]:
train_df = shuffle(train_df)
train_df.head()

,sentence,BookRestaurant,GetWeather,PlayMusic,RateBook
256,book a spot for 1 close to geraldine's house,1,0,0,0
3696,What is the weather in Deeth,0,1,0,0
5490,play music by Rodney Whitaker,0,0,1,0
5222,play top-twenty song from 2015,0,0,1,0
1952,I want to book a restaurant in Barbados that s...,1,0,0,0


### 4. Now saving the dataframe column(X_train,Y_train) as a numpy array

Here,
- X_train -> Its a feature(INPUT)
- Y_tain -> Its a label(OUTPUT)

In [135]:
X_train = train_df["sentence"].fillna("fillna").values
Y_train = train_df[["BookRestaurant", "GetWeather", "PlayMusic", "RateBook"]].values

In [136]:
print("Shape of X_train:",X_train.shape)
print("Shape of Y_train:",Y_train.shape)

Shape of X_train: (7929,)
Shape of Y_train: (7929, 4)


In [137]:
print("-------------------------First 2 rows of X_train numpy array--------------------")
for i in range(0,2):
    print(i,'.',X_train[i])

-------------------------First 2 rows of X_train numpy array--------------------
0 . book a spot for 1 close to geraldine's house
1 . What is the weather in Deeth


### 7. Setting up Tokenizer class
The Tokenizer class in Keras has various methods which help to prepare text so it can be used in neural network models.

In [138]:
Tokenizer = Tokenizer()
print(X_train[0]) # training dataset 1st sentence

book a spot for 1 close to geraldine's house


In [139]:
print("(Input->Sentence) Length of X_train:",X_train.shape) # Input -> Input
print("(output->Labels) Length of Y_train:",Y_train.shape) # output -> Labels
texts = X_train
print(texts[0])

(Input->Sentence) Length of X_train: (7929,)
(output->Labels) Length of Y_train: (7929, 4)
book a spot for 1 close to geraldine's house


In [140]:
Tokenizer.fit_on_texts(texts) 
Tokenizer_vocab_size = len(Tokenizer.word_index) + 1
print("Tokenizer vocabulary size:",Tokenizer_vocab_size)

Tokenizer vocabulary size: 7526


### 8. Now find the length of longest sublist in X_train,X_test and set it as max Word Count

In [141]:
len(max(X_train,key=len))

186

In [142]:
maxWordCount= 5500
maxDictionary_size=Tokenizer_vocab_size

In [143]:
print("(Input->Sentence) Length of X_train:",X_train.shape) # Input -> Input
print("(output->Labels) Length of Y_train:",Y_train.shape) # output -> Labels

(Input->Sentence) Length of X_train: (7929,)
(output->Labels) Length of Y_train: (7929, 4)


### 9. Now setting up data for validation and training:

In [144]:
num_test_samples = 1586 # Test samples for validation


# Phase 1: Setting up data for training
X_train = X_train[num_test_samples:] # 1586 samples to n ----> Sentence (Input)
Y_train = Y_train[num_test_samples:] # 1586 samples to n ----> Labels (Output)

# Phase 2: Setting up data for validation
X_val = X_train[:num_test_samples] # First 1586 Samples --> Sentence (Input)
Y_val = Y_train[:num_test_samples] # First 1586 Samples --> Labels (Output)

In [123]:
print("(Input->Sentence) Length of X_train:",X_train.shape) # Input -> Input
print("(output->Labels) Length of Y_train:",Y_train.shape) # output -> Labels

(Input->Sentence) Length of X_train: (6343,)
(output->Labels) Length of Y_train: (6343, 4)


###  11.  Encoding Operation--->Turn text into a numerical array(using Tokenizer.texts_to_sequences)--->Uses Tokenizer_word_index.


In [145]:
# Phase 3: Encoding Operation--->Turn text into a numerical array(using Tokenizer.texts_to_sequences)--->Uses Tokenizer_word_index.
X_train_encoded_words = Tokenizer.texts_to_sequences(X_train)
X_val_encoded_words = Tokenizer.texts_to_sequences(X_val)

In [146]:

print("(output->Labels) Length of Y_train:",Y_train.shape) # output -> Labelsprint("(Input->Sentence) Length of X_train:",len(X_train_encoded_words)) # Input -> Input

(output->Labels) Length of Y_train: (6343, 4)


### 12. Padding all text to same size

In [147]:
X_train_encoded_padded_words = sequence.pad_sequences(X_train_encoded_words, maxlen=maxWordCount)
X_val_encoded_padded_words = sequence.pad_sequences(X_val_encoded_words, maxlen=maxWordCount)

In [148]:
print("(Input->Sentence) Length of X_train:",X_train_encoded_padded_words.shape) # Input -> Input
print("(output->Labels) Length of Y_train:",Y_train.shape) # output -> Labels

(Input->Sentence) Length of X_train: (6343, 5500)
(output->Labels) Length of Y_train: (6343, 4)


In [149]:
print(Y_train.shape)
print(Y_val.shape)

(6343, 4)
(1586, 4)


### 13. Now defining the neural network model

In [150]:
#model
model = Sequential()

model.add(Embedding(maxDictionary_size, 32, input_length=maxWordCount)) #to change words to ints
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.5))
# model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
 #hidden layers
model.add(LSTM(10))
# model.add(Flatten())
model.add(Dropout(0.6))
model.add(Dense(1200, activation='relu',W_constraint=maxnorm(1)))
# model.add(Dropout(0.6))
model.add(Dense(500, activation='relu',W_constraint=maxnorm(1)))

# model.add(Dropout(0.5))
 #output layer
model.add(Dense(4, activation='softmax'))

# Compile model
# adam=Adam(lr=learning_rate, beta_1=0.7, beta_2=0.999, epsilon=1e-08, decay=0.0000001)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 5500, 32)          240832    
_________________________________________________________________
lstm_3 (LSTM)                (None, 10)                1720      
_________________________________________________________________
dropout_3 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1200)              13200     
_________________________________________________________________
dense_8 (Dense)              (None, 500)               600500    
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 2004      
Total params: 858,256
Trainable params: 858,256
Non-trainable params: 0
_________________________________________________________________


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1200, activation="relu", kernel_constraint=<keras.con...)`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(500, activation="relu", kernel_constraint=<keras.con...)`
  app.launch_new_instance()


### 14. Now setting up learning rate,optimizers,batch_size, and compiling the model

In [151]:
learning_rate=0.0001
epochs = 25
batch_size = 32 #32
sgd = SGD(lr=learning_rate, nesterov=True, momentum=0.7, decay=1e-4)
Nadam = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
model.compile(loss='categorical_crossentropy', optimizer=Nadam, metrics=['accuracy'])

In [152]:
print(X_train_encoded_padded_words.shape)
print(Y_train.shape)

print(X_val_encoded_padded_words.shape)
print(Y_val.shape)

(6343, 5500)
(6343, 4)
(1586, 5500)
(1586, 4)


### 15. Training the model

In [ ]:
# model.fit(Train_input,Train_output,epochs,batch_size,verbose,validation_data=(val_input,val_output))
history  = model.fit(X_train_encoded_padded_words,Y_train, epochs = epochs, batch_size=batch_size, verbose=1,
                    validation_data=(X_val_encoded_padded_words, Y_val))

Train on 6343 samples, validate on 1586 samples
Epoch 1/25
6343/6343 [==============================] - 430s 68ms/step - loss: 0.3403 - acc: 0.8490 - val_loss: 0.0367 - val_acc: 0.9893
Epoch 2/25
6343/6343 [==============================] - 464s 73ms/step - loss: 0.1060 - acc: 0.9600 - val_loss: 0.0087 - val_acc: 0.9968
Epoch 3/25
1888/6343 [=======>......................] - ETA: 4:55 - loss: 0.0793 - acc: 0.9709

### 16. Calculating the score

In [ ]:
print("=============================== Score =========================================")
# Finally calucating the score.
score = model.evaluate(X_val_encoded_padded_words, Y_val, verbose=1)
print('Test accuracy:', score[1],'%')

### 17. Predicting the text

In [ ]:
phrase = "Book The Oriel in Allison for a party of four."
tokens = Tokenizer.texts_to_sequences([phrase])
tokens = pad_sequences(tokens, maxlen=5500)
prediction = model.predict(np.array(tokens))
i,j = np.where(prediction == prediction.max()) #calculates the index of the maximum element of the array across all axis
# i->rows, j->columns
i = int(i)
j = int(j)
print(prediction)
total_possible_outcomes = ['1_BookRestaurant','2_GetWeather','3_PlayMusic','4_RateBook']
print("Result:",total_possible_outcomes[j])


### 18. Saving your model to json and weights as HDF5 format

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("./data/model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("./data/model.h5")
print("Saved model to disk")

### 19. Storing Tokenizer for storing

In [ ]:
# saving
with open('./data/tokenizer.pickle', 'wb') as handle:
    pickle.dump(Tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)